In [1]:
# import matplotlib.pyplot as plt
import pandas as pd
# import numpy as np
import requests
# import time
# import scipy.stats as st
# from scipy import stats 
# from scipy.stats import linregress
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

In [2]:
# Create a variable for the url
census_url = "https://services.arcgis.com/fLeGjb7u4uXqeF9q/arcgis/rest/services/Census_Tracts_2010/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [3]:
# Retrieve json data using requests
response = requests.get(census_url).json()

In [4]:
# Create a variable to hold json output
census_tracts_data = response['features']
census_tracts_data

[{'attributes': {'OBJECTID': 1,
   'STATEFP10': '42',
   'COUNTYFP10': '101',
   'TRACTCE10': '009400',
   'GEOID10': '42101009400',
   'NAME10': '94',
   'NAMELSAD10': 'Census Tract 94',
   'MTFCC10': 'G5020',
   'FUNCSTAT10': 'S',
   'ALAND10': 366717,
   'AWATER10': 0,
   'INTPTLAT10': '+39.9632709',
   'INTPTLON10': '-075.2322437',
   'LOGRECNO': '10429'},
  'geometry': {'rings': [[[2675024.65085323, 238565.796414465],
     [2674877.64392939, 238584.840011552],
     [2674747.49687998, 238601.878035218],
     [2674591.31838655, 238621.469859555],
     [2674482.96656914, 238634.665043131],
     [2674289.67332056, 238659.424179971],
     [2674012.8405409, 238697.427384794],
     [2673816.91442356, 238724.638616472],
     [2673651.36357357, 238745.498154804],
     [2673446.39515148, 238774.292716637],
     [2672900.60687298, 238846.14231047],
     [2672931.04972547, 239073.876763135],
     [2672939.28035206, 239144.090533465],
     [2672951.57429072, 239229.692724466],
     [2672975.94

In [49]:
# Create empty lists to hold information
GEOID = []
Lat = []
Lng = []
Geometry = []


# Loop through each record in json data and append data to relevant list
for info in census_tracts_data:
    GEOID.append(info['attributes']['GEOID10'])
    Lat.append(info['attributes']['INTPTLAT10'])
    Lng.append(info['attributes']['INTPTLON10'])
    Geometry.append(info['geometry']['rings'][0])

In [50]:
# Create a DataFrame to store information
census_tracts_summary_df = pd.DataFrame({"GEOID": GEOID, "Latitude": Lat,\
                                         "Longitude": Lng, "Tract Coordinates": Geometry})
# census_tracts_summary_df = pd.DataFrame({"GEOID": GEOID, "Latitude": Lat,\
#                                          "Longitude": Lng})
census_tracts_summary_df

GEOID     Latitude     Longitude  \
0    42101009400  +39.9632709  -075.2322437   
1    42101009500  +39.9658709  -075.2379140   
2    42101009600  +39.9655396  -075.2435075   
3    42101013800  +39.9764504  -075.1771771   
4    42101013900  +39.9750563  -075.1711846   
..           ...          ...           ...   
379  42101037200  +39.9129487  -075.1599379   
380  42101038300  +40.0108630  -075.1248984   
381  42101039000  +40.0357300  -075.0938840   
382  42101037800  +39.9805802  -075.0956845   
383  42101037700  +39.9824381  -075.1506932   

                                     Tract Coordinates  
0    [[2675024.65085323, 238565.796414465], [267487...  
1    [[2673234.36079864, 241420.841229305], [267323...  
2    [[2671038.04891831, 239090.919051886], [267086...  
3    [[2690492.95554648, 245298.126453713], [269057...  
4    [[2690571.26083606, 245288.387628049], [269049...  
..                                                 ...  
379  [[2691718.7027669, 223096.792247549], [2691739...  
380  [[2706067.28857099, 260203.892901972], [270597...  
381  [[2713664.44266798, 268779.110659629], [271362...  
382  [[2708201.33154698, 242805.601911217], [270754...  
383  [[2696488.10202073, 248359.091788471], [269670...  

[384 rows x 4 columns]

In [51]:
# Save information in a csv file
census_tracts_summary_df.to_csv("census_tracts_summary.csv", encoding="utf-8", index=False)

In [52]:
engine = create_engine('sqlite:///philly.sqlite', echo=True)
sqlite_connection = engine.connect()

2021-05-01 10:04:45,956 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-05-01 10:04:45,957 INFO sqlalchemy.engine.base.Engine ()
2021-05-01 10:04:45,961 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-05-01 10:04:45,961 INFO sqlalchemy.engine.base.Engine ()


In [53]:
# Add primary key to "merged" table that contains all cleaned data
sqlite_connection.execute('DROP TABLE "censustracks"' )
sqlite_connection.execute('CREATE TABLE "censustracks" ( "index" INTEGER PRIMARY KEY, "GEOID" TEXT, "Latitude" TEXT, "Longitude" TEXT, "Tract Coordinates" VARCHAR)' )

2021-05-01 10:04:46,858 INFO sqlalchemy.engine.base.Engine DROP TABLE "censustracks"
2021-05-01 10:04:46,863 INFO sqlalchemy.engine.base.Engine ()
2021-05-01 10:04:46,867 INFO sqlalchemy.engine.base.Engine COMMIT
2021-05-01 10:04:46,868 INFO sqlalchemy.engine.base.Engine CREATE TABLE "censustracks" ( "index" INTEGER PRIMARY KEY, "GEOID" TEXT, "Latitude" TEXT, "Longitude" TEXT, "Tract Coordinates" VARCHAR)
2021-05-01 10:04:46,869 INFO sqlalchemy.engine.base.Engine ()
2021-05-01 10:04:46,872 INFO sqlalchemy.engine.base.Engine COMMIT


In [54]:
sqlite_table = "censustracks"
census_tracts_summary_df.to_sql(sqlite_table, sqlite_connection, if_exists = 'replace', index= True)

2021-05-01 10:04:48,259 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("censustracks")
2021-05-01 10:04:48,260 INFO sqlalchemy.engine.base.Engine ()
2021-05-01 10:04:48,265 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("censustracks")
2021-05-01 10:04:48,266 INFO sqlalchemy.engine.base.Engine ()
2021-05-01 10:04:48,266 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-05-01 10:04:48,267 INFO sqlalchemy.engine.base.Engine ()
2021-05-01 10:04:48,268 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("censustracks")
2021-05-01 10:04:48,268 INFO sqlalchemy.engine.base.Engine ()
2021-05-01 10:04:48,269 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'censustracks' AND type = 'table'
2021-05-01 10:04:48,269 INFO sqlalchemy.engine.base.Engine ()
2021-05-01 10:04:48,271 INFO sqlalchemy.engine.base.Engine PRAGMA main.fore

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 4 - probably unsupported type.
[SQL: INSERT INTO censustracks ("index", "GEOID", "Latitude", "Longitude", "Tract Coordinates") VALUES (?, ?, ?, ?, ?)]
[parameters: ((0, '42101009400', '+39.9632709', '-075.2322437', [[2675024.65085323, 238565.796414465], [2674877.64392939, 238584.840011552], [2674747.49687998, 238601.878035218], [2674591.31838655, 238621.469859555 ... (1062 characters truncated) ... [2675347.8811779, 239675.301085219], [2675273.56735015, 239118.840255886], [2675198.28009914, 238544.600590721], [2675024.65085323, 238565.796414465]]), (1, '42101009500', '+39.9658709', '-075.2379140', [[2673234.36079864, 241420.841229305], [2673236.33553223, 241397.747115389], [2673197.36940281, 241090.177192464], [2673179.86090764, 240955.747999385 ... (1025 characters truncated) ... 2673152.93018723, 241894.115217134], [2673178.55644831, 241748.029551297], [2673203.32083447, 241602.998345301], [2673234.36079864, 241420.8412

In [ ]:
sqlite_connection.close()